In [ ]:
import json
import os
import math

import utils.instrument as instru

with open('./config/scpi_commands.json') as f:
    scpi_cmds = json.load(f)['scpi_cmds']

ip_addr = r'192.168.0.1'

sa = instru.SpectrumAnalyzer(scpi_cmds, ip_addr)

Connected to: Rohde&Schwarz,FSQ-26,200750/026,4.75 successfully.


In [6]:
def unit_measurement(sa: instru.SpectrumAnalyzer, sa_config: dict):
    sa.config(param='continues_sweep', value='OFF')  # stop sweep
    input('Waiting for board launch up. Press enter to start measurement.')

    sa.config(param=sa_config)  # start continues sweep

    input(f"Waiting trace to stabilize. Press Enter to stop sweep.")
    sa.config(param='continues_sweep', value='OFF')  # stop sweep

    peak_freq, peak_power = sa.aquire_peak_point()

    return peak_freq, peak_power

In [ ]:
def search(sa: instru.SpectrumAnalyzer, giteki_dict: dict, freq_band: str, measure_item: str, rule='49_27_3', method='general'):
    """
    freq_band (str): e.g., 7.587GHz~8.4GHz
    measure_item (str): peak or average
    """
    if freq_band == giteki_dict['masks'][rule]['occupied_freq_band']:
        occupied_freq_band_flag = True
    else:
        occupied_freq_band_flag = False

    # search step
    # construct spectrum analyzer config
    sa_cfg = giteki_dict['search']['common']
    if occupied_freq_band_flag and measure_item == 'peak':
        sa_cfg.update(giteki_dict['search']['diff']['peak']['general'])

    # parse freq. band
    start_freq, stop_freq = freq_band.split('~')[0], freq_band.split('~')[1]
    sa_cfg.update({'start_freq': start_freq, 'stop_freq': stop_freq})

    # perform a unit measurement
    search_freq, search_peak = unit_measurement(sa, sa_cfg)
    # save screenshot
    screenshot = f'search_{measure_item}_{freq_band}.png'
    sa.save_screenshot(screenshot)

    if measure_item == 'average':
        trace_file = f'{freq_band}_trace.csv'
        sa.save_trace_to_csv(trace_file)

    if not occupied_freq_band_flag:
        # for spurious measurement, we need to compare with limit to determin if go to zoom in step
        limit = giteki_dict['masks'][rule][freq_band][measure_item]
        if measure_item == 'peak' and (limit - search_peak) >= 3:
            return search_freq, search_peak
        elif measure_item == 'average' and (limit >= search_peak):
            return search_freq, search_peak
        
    # zoom in step
    if occupied_freq_band_flag and measure_item == 'peak':
        spans = ['100MHz']
        sa_cfg.update(giteki_dict['search']['diff']['peak'][method])
    else:
        spans = ['100MHz', '10MHz']

    # re-construct spectrum analyzer config
    del sa_cfg['start_freq']
    del sa_cfg['stop_freq']
    for span in spans:
        sa_cfg.update({'span': span, 'center_freq': search_freq})

        # perform a unit measurement
        search_freq, search_peak = unit_measurement(sa, sa_cfg)
        # save screenshot
        screenshot = f'search_{measure_item}_{freq_band}_{span}_{method}.png'
        sa.save_screenshot(screenshot)

    return search_freq, search_peak

In [ ]:
def if_spurious(freq, giteki_dict: dict, rule) -> bool:
    occupied_freq_band = giteki_dict['masks'][rule]['occupied_freq_band']
    start_freq = occupied_freq_band.split('~')[0].split('GHz')[0] * 1e9
    stop_freq = occupied_freq_band.split('~')[1].split('GHz')[0] * 1e9

    if start_freq <= freq <= stop_freq:
        return False
    return True

import utils.components as comps

def measure(sa: instru.SpectrumAnalyzer, giteki_dict: dict, center_freq: float, rule='49_27_3', method='general'):
    spurious_flag = if_spurious(center_freq, giteki_dict, rule)

    if not spurious_flag and method == 'exception':
        sa_cfg = giteki_dict['measure']['diff']['ave']['exception']
    else:
        sa_cfg = giteki_dict['measure']['common']
    sa_cfg.update({'center_freq': center_freq})

    # perform a unit measurement
    measure_freq, measure_ave = unit_measurement(sa, sa_cfg)
    # save screenshot
    screenshot = f'measure_{method}_{round(center_freq / 1e9, 5)}GHz.png'
    sa.save_screenshot(screenshot)

    cal_ave = None
    if spurious_flag or method == 'exception':
        trace_file = f'measure_{round(center_freq / 1e9, 5)}GHz.csv'
        sa.save_trace_to_csv(trace_file)

        cal = comps.TraceDataCalculator(trace_file)

        if spurious_flag:
            cal_ave = cal.calculate_ave_spurious()
        else:
            cal_ave = cal.calculate_ave_power()
    return measure_freq, measure_ave, cal_ave
    

In [ ]:
def compliance_decision(result: tuple, measure_item: str, freq_band: str, masks: dict):
    """
    result (tuple): search_freq, search_peak
    measure_item (str): peak or average
    freq_band (str): e.g., 7.587GHz~8.4GHz
    masks (dict): limitation dict
    """
    